In [1]:
import psycopg2
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

host="13.43.84.151"
dbname="postgres"
user="ping-fan.chen.23@ucl.ac.uk"
password="khsLcR"
port="5432"
schema="schema_pingfanchen23uclacuk"

In [2]:
conn = psycopg2.connect (
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
print("Database connection established")

Database connection established


In [3]:
# fetch the java driver for postgress to allow Spark to connect to postgress
!curl -o postgresql-42.3.2.jar https://jdbc.postgresql.org/download/postgresql-42.3.2.jar
# uses the curl command to download the PostgreSQL JDBC driver from the official site.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1015k  100 1015k    0     0   659k      0  0:00:01  0:00:01 --:--:--  659k


In [5]:
import os
current_dir = os.getcwd() # assigns the current working directory to the variable current_dir
print(current_dir)

jar_location = current_dir + "/postgresql-42.3.2.jar"
# Constructs the file path for the PostgreSQL JDBC driver JAR file by appending the filename to the current directory path
jar_location

/home/jovyan/dataengineer


'/home/jovyan/dataengineer/postgresql-42.3.2.jar'

In [6]:
from pyspark.sql import SparkSession
# Initialize Spark Session
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", jar_location)\
    .appName("Enhanced Data Warehouse ETL") \
    .getOrCreate()
# Sets the configuration property spark.driver.extraClassPath to the path of the JDBC driver.
# This tells Spark where to find the driver for the database connection.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 18:34:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, lit, lead

# Read the CSV file into a Spark DataFrame
df = spark.read.csv("df_states_zip.csv", header=True, inferSchema=True)
# Show the CSV file before cleaning 
df.show()

+---+--------+--------------+-----+-----------------+---------------+----------+-------+--------+
|_c0|zip_code|          city|state|           county|       timezone|area_codes|    lat|     lon|
+---+--------+--------------+-----+-----------------+---------------+----------+-------+--------+
|  0|   35004|         Moody|   AL| St. Clair County|America/Chicago|   ['205']|33.6034|-86.4944|
|  1|   35005|    Adamsville|   AL| Jefferson County|America/Chicago|   ['205']|33.5929| -86.994|
|  2|   35006|         Adger|   AL| Jefferson County|America/Chicago|   ['205']|33.4462| -87.223|
|  3|   35007|     Alabaster|   AL|    Shelby County|America/Chicago|   ['205']|33.2187|-86.7835|
|  4|   35010|Alexander City|   AL|Tallapoosa County|America/Chicago|   ['256']|32.9011|-85.9178|
|  5|   35011|Alexander City|   AL|Tallapoosa County|America/Chicago|   ['256']|32.9394|-85.9466|
|  6|   35013|       Allgood|   AL|    Blount County|America/Chicago|   ['205']|33.9119|-86.5094|
|  7|   35014|      

24/03/03 18:37:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , zip_code, city, state, county, timezone, area_codes, lat, lon
 Schema: _c0, zip_code, city, state, county, timezone, area_codes, lat, lon
Expected: _c0 but found: 
CSV file: file:///home/jovyan/dataengineer/df_states_zip.csv


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window

# Read the CSV file into a Spark DataFrame
median_value_df = spark.read.csv("median_value.csv", header=True, inferSchema=True)
median_value_df.show()

# Define a window spec to look ahead by one row
windowSpec = Window.orderBy(lit(0))

# Shift the 'Median value (dollars)' column up by one row
median_value_df = median_value_df.withColumn("shifted_median_value", lag("Median value (dollars)").over(windowSpec))

# Filter out the 'Estimate' rows and keep the rows with state names
# Then drop the original 'Median value (dollars)' column
# Lastly, remove rows where the 'shifted_median_value' is null
median_value_clean = median_value_df.filter(~col("Label (Grouping)").contains("Estimate")) \
                                    .drop("Median value (dollars)") \
                                    .filter(col("shifted_median_value").isNotNull()) \
                                    .withColumnRenamed("shifted_median_value", "Median value (dollars)") \
                                    .withColumnRenamed("Label (Grouping)", "State")

# Show the cleaned DataFrame
median_value_clean.show()

+--------------------+----------------------+
|    Label (Grouping)|Median value (dollars)|
+--------------------+----------------------+
|             Alabama|                  NULL|
|            Estimate|               200,900|
|              Alaska|                  NULL|
|            Estimate|               336,900|
|             Arizona|                  NULL|
|            Estimate|               402,800|
|            Arkansas|                  NULL|
|            Estimate|               179,800|
|          California|                  NULL|
|            Estimate|               715,900|
|            Colorado|                  NULL|
|            Estimate|               531,100|
|         Connecticut|                  NULL|
|            Estimate|               347,200|
|            Delaware|                  NULL|
|            Estimate|               337,200|
|District of Columbia|                  NULL|
|            Estimate|               711,100|
|             Florida|            

24/03/03 09:33:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------------------+
|               State|Median value (dollars)|
+--------------------+----------------------+
|              Alaska|               200,900|
|             Arizona|               336,900|
|            Arkansas|               402,800|
|          California|               179,800|
|            Colorado|               715,900|
|         Connecticut|               531,100|
|            Delaware|               347,200|
|District of Columbia|               337,200|
|             Florida|               711,100|
|             Georgia|               354,100|
|              Hawaii|               297,400|
|               Idaho|               820,100|
|            Illinois|               432,500|
|             Indiana|               251,600|
|                Iowa|               208,700|
|              Kansas|               194,600|
|            Kentucky|               206,600|
|           Louisiana|               196,300|
|               Maine|            

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window


# Read the CSV file into a Spark DataFrame
lower_value_df = spark.read.csv('lower_value.csv', header=True, inferSchema=True)

# Show the initial state of the DataFrame
lower_value_df.show()

# Define a window spec to look ahead by one row
windowSpec = Window.orderBy(lit(0))

# Shift the 'Lower value (dollars)' column up by one row
lower_value_df = lower_value_df.withColumn("shifted_lower_value", lag("Lower value quartile (dollars)").over(windowSpec))

# Filter out the 'Estimate' rows and keep the rows which have the state names 
# Next, drop the original 'Lower value (dollars)' column
# Lastly, remove rows where the 'shifted_lower_value' is null
lower_value_clean = lower_value_df.filter(~col("Label (Grouping)").contains("Estimate")) \
                                  .drop("Lower value quartile (dollars)") \
                                  .filter(col("shifted_lower_value").isNotNull()) \
                                  .withColumnRenamed("shifted_lower_value", "Lower value quartile (dollars)") \
                                  .withColumnRenamed("Label (Grouping)", "State")

# Show the cleaned DataFrame
lower_value_clean.show()


+--------------------+------------------------------+
|    Label (Grouping)|Lower value quartile (dollars)|
+--------------------+------------------------------+
|             Alabama|                          NULL|
|            Estimate|                       105,000|
|              Alaska|                          NULL|
|            Estimate|                       232,200|
|             Arizona|                          NULL|
|            Estimate|                       262,500|
|            Arkansas|                          NULL|
|            Estimate|                        98,300|
|          California|                          NULL|
|            Estimate|                       460,900|
|            Colorado|                          NULL|
|            Estimate|                       366,900|
|         Connecticut|                          NULL|
|            Estimate|                       240,600|
|            Delaware|                          NULL|
|            Estimate|      

24/03/03 09:33:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------------------------------+
|               State|Lower value quartile (dollars)|
+--------------------+------------------------------+
|              Alaska|                       105,000|
|             Arizona|                       232,200|
|            Arkansas|                       262,500|
|          California|                        98,300|
|            Colorado|                       460,900|
|         Connecticut|                       366,900|
|            Delaware|                       240,600|
|District of Columbia|                       228,600|
|             Florida|                       468,200|
|             Georgia|                       222,500|
|              Hawaii|                       175,300|
|               Idaho|                       513,300|
|            Illinois|                       293,700|
|             Indiana|                       148,100|
|                Iowa|                       126,400|
|              Kansas|      

24/03/03 09:33:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [9]:
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema={schema}"

# Connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

# Write the cleaned median value DataFrame to PostgreSQL
median_value_clean.write.jdbc(url=url, table="median_value_table", mode="overwrite", properties=properties)

# Write the cleaned upper value DataFrame to PostgreSQL
upper_value_clean.write.jdbc(url=url, table="upper_value_table", mode="overwrite", properties=properties)

# Write the cleaned lower value DataFrame to PostgreSQL
lower_value_clean.write.jdbc(url=url, table="lower_value_table", mode="overwrite", properties=properties)

24/03/03 09:33:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 09:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 0

We can execute a simple SQL query to **combine the tables** of upper, median, and lower quartile estimates using state names as the key. After querying, this consolidated Spark dataframe can then be stored in our Postgres database.

In [22]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Data Joining").getOrCreate()

# Given the `upper_value_clean`, `median_value_clean`, and `lower_value_clean` data frame we already defined above
# Let's register these DataFrames as temporary views
upper_value_clean.createOrReplaceTempView("upper_value_view")
median_value_clean.createOrReplaceTempView("median_value_view")
lower_value_clean.createOrReplaceTempView("lower_value_view")

# Perform the SQL join query using state as the key, combining the upper,median,lower quartile values together 
join_query = """
SELECT 
  u.State, 
  u.`Upper value quartile (dollars)` AS Upper_Value,
  m.`Median value (dollars)` AS Median_Value,
  l.`Lower value quartile (dollars)` AS Lower_Value
FROM 
  upper_value_view u
JOIN 
  median_value_view m 
ON 
  u.State = m.State
JOIN 
  lower_value_view l
ON 
  u.State = l.State
"""

# Execute the query
combined_quartile_df = spark.sql(join_query)

# Show the results
combined_quartile_df.show()

# Write the cleaned quartile value DataFrame to PostgreSQL
combined_quartile_df.write.jdbc(url=url, table="combined_value_quartile", mode="overwrite", properties=properties)

24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 1

+--------------------+-----------+------------+-----------+
|               State|Upper_Value|Median_Value|Lower_Value|
+--------------------+-----------+------------+-----------+
|              Alaska|    331,200|     200,900|    105,000|
|             Arizona|    455,200|     336,900|    232,200|
|            Arkansas|    602,700|     402,800|    262,500|
|          California|    290,700|     179,800|     98,300|
|            Colorado|  1,083,100|     715,900|    460,900|
|         Connecticut|    737,500|     531,100|    366,900|
|            Delaware|    504,900|     347,200|    240,600|
|District of Columbia|    468,200|     337,200|    228,600|
|             Florida|  1,057,600|     711,100|    468,200|
|             Georgia|    524,200|     354,100|    222,500|
|              Hawaii|    450,200|     297,400|    175,300|
|               Idaho|  1,133,500|     820,100|    513,300|
|            Illinois|    638,200|     432,500|    293,700|
|             Indiana|    383,500|     2

24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:08:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 1

Next we’ll import the **fourth csv table from ACS**, detailing the estimated number of houses in each state, categorized by price ranges, ranging from less than 50,000 USD to more than 1,000,000 USD. Given that this table is larger and more complicated, we utilize pandas for cleaning and then convert the cleaned pandas dataframe into a Spark dataframe. Finally, we store the spark dataframe in the Postgres database. 

In [11]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file_path = 'num_of_houses_by_price_range.csv'
df = pd.read_csv(csv_file_path)
df.head()

# Now we can do some data cleaning: 
# Rename the first column name to "State"
df.rename(columns={df.columns[0]: 'State'}, inplace=True)

# Remove the "Total:!!" prefix from the column names starting from the third column
df.columns = [col if idx < 2 else col.replace('Total:!!', '') for idx, col in enumerate(df.columns)]

# Shift all the data points one step upward in columns other than 'State'
for column in df.columns[1:]: #  iterate over each column, except 'State', and shift them
    df[column] = df[column].shift(-1)

# Drop all rows which contains NaN value 
df_cleaned = df.dropna()

# Display the cleaned DataFrame
df_cleaned.head()

# Convert the pandas DataFrame to the Spark DataFrame
spark_df_num_of_houses = spark.createDataFrame(df_cleaned)
spark_df_num_of_houses.show()

# Write the Spark DataFrame to the PostgreSQL database
spark_df_num_of_houses.write.jdbc(url=url, table="num_of_houses_by_price_range", mode='overwrite', properties=properties)


,Label (Grouping),Total:,"Total:!!Less than $50,000","Total:!!$50,000 to $99,999","Total:!!$100,000 to $149,999","Total:!!$150,000 to $199,999","Total:!!$200,000 to $299,999","Total:!!$300,000 to $499,999","Total:!!$500,000 to $999,999","Total:!!$1,000,000 or more"
0,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estimate,"1,416,333","147,494","186,145","179,651","192,181","298,254","278,735","115,476","18,397"
2,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Estimate,"181,586","10,015","6,103","7,964","11,081","38,637","75,646","29,463","2,677"
4,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,State,Total:,"Less than $50,000","$50,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 to $299,999","$300,000 to $499,999","$500,000 to $999,999","$1,000,000 or more"
0,Alabama,"1,416,333","147,494","186,145","179,651","192,181","298,254","278,735","115,476","18,397"
2,Alaska,"181,586","10,015","6,103","7,964","11,081","38,637","75,646","29,463","2,677"
4,Arizona,"1,921,746","115,424","67,678","57,890","80,092","265,323","692,276","541,367","101,696"
6,Arkansas,"808,558","82,513","123,326","109,924","129,888","175,545","129,125","47,983","10,254"
8,California,"7,565,502","193,507","136,690","103,683","110,071","331,993","1,297,381","3,253,409","2,138,768"


+--------------------+---------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|               State|   Total:|Less than $50,000|$50,000 to $99,999|$100,000 to $149,999|$150,000 to $199,999|$200,000 to $299,999|$300,000 to $499,999|$500,000 to $999,999|$1,000,000 or more|
+--------------------+---------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|             Alabama|1,416,333|          147,494|           186,145|             179,651|             192,181|             298,254|             278,735|             115,476|            18,397|
|              Alaska|  181,586|           10,015|             6,103|               7,964|              11,081|              38,637|              75,646|              29,463|             2,677|
|             Arizona|1,921,74

Finally let's print out **all existing tables** in our Postgres database.

In [23]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password, port=port)
# Create Cursor object 
cur = conn.cursor()

# Query to fetch table names in the specified schema
cur.execute(f"""
SELECT table_name FROM information_schema.tables
WHERE table_schema = '{schema}' AND table_type = 'BASE TABLE';
""")

# Fetch all table names
tables = cur.fetchall()

# Iterate over table names and select data from each table
for table in tables:
    table_name = table[0]
    print(f"Data from table: {table_name}")
    
    # Fetch data from each table
    query = f"SELECT * FROM {schema}.{table_name};"
    df = pd.read_sql_query(query, conn)
    
    # Display the DataFrame
    print(df.head())  # Adjust as per your need
    
# Close the cursor and connection
cur.close()
conn.close()


Data from table: people
   id      name      city profession
0   1  John Doe  New York   Engineer
1   2  John Doe  New York   Engineer
2  34  John Doe  New York   Engineer
3  35  John Doe  New York   Engineer
4  36  John Doe  New York   Engineer
Data from table: num_of_houses_by_price_range
        State     Total: Less than $50,000 $50,000 to $99,999  \
0     Alabama  1,416,333           147,494            186,145   
1      Alaska    181,586            10,015              6,103   
2     Arizona  1,921,746           115,424             67,678   
3    Arkansas    808,558            82,513            123,326   
4  California  7,565,502           193,507            136,690   

  $100,000 to $149,999 $150,000 to $199,999 $200,000 to $299,999  \
0              179,651              192,181              298,254   
1                7,964               11,081               38,637   
2               57,890               80,092              265,323   
3              109,924              129,888  

/tmp/ipykernel_95/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_95/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_95/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_95/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBA

In [19]:
# To use the latest version of duckdb
!pip install duckdb

# Assuming you have DuckDB installed and the duckdb Python package
import duckdb

# Connect to DuckDB
con = duckdb.connect()


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [27]:
# Save the combined DataFrame as a Parquet file
combined_quartile_df.write.parquet("combined_value_quartile.parquet", mode="overwrite")

# Load the Parquet file into DuckDB as a table
con.execute("CREATE TABLE combined_value_quartile AS SELECT * FROM read_parquet('/home/jovyan/DE w6 workshops/combined_value_quartile.parquet/part-00000-eb3d1bcb-b705-4c99-8f52-25785be2e534-c000.snappy.parquet')")

# Perform a query in DuckDB to verify the data is loaded
result = con.execute("SELECT * FROM combined_value_quartile LIMIT 10").fetchall()

# Print the result
print(result)

24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 10:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/03 1

IOException: IO Error: No files found that match the pattern "/home/jovyan/DE w6 workshops/combined_value_quartile.parquet/part-00000-eb3d1bcb-b705-4c99-8f52-25785be2e534-c000.snappy.parquet"

In [26]:
# Save the spark_df_num_of_houses DataFrame as a Parquet file
spark_df_num_of_houses.write.parquet("num_of_houses.parquet", mode="overwrite")

# Load the Parquet file into DuckDB as a table
con.execute("CREATE TABLE num_of_houses AS SELECT * FROM read_parquet('num_of_houses.parquet')")

# Perform a query in DuckDB to verify the data is loaded
result = con.execute("SELECT * FROM num_of_houses LIMIT 10").fetchall()

# Print the result
print(result)


IOException: IO Error: No files found that match the pattern "num_of_houses.parquet"